# Completeness Motivation

In astrophysics we are limited by the resolution of our telescope, the duration of our exposure, and the quality of our data reduction pipeline. If there is an absorber cloud present on the outskirts of the glaaxy's gravitational potential well, but it our signal to noise ratio (SNR) is insufficent to identify it in our spectrum of the quasar, we will not have a detection in our catalog. More formally, if there is an abosrber whose restframe equaivlent width is $W_{r}$ and our sptrum has a $3\sigma$ deftection limit threshold of $3\sigma = W_{min}$, and $W_{r} < W_{min}$ we cannot detect it. However, there are statistic methods that allow us to quanity, based off the SNR of our data, what percentage of absorbers we miss for a given $W_{r}$ threshold we are searching for; this is a completeness test.

In the literature there are two main methods for performing completeness tests. The first involves taking quasar spectra and inserting at known points synthetic absorbers of known $W_{r}$. Then an absorber doublet searching algorithm is ran and the percentage of recovered abosrbers is compared to the total number inserted as a function of $W_{r}$. This method is simple to understand and is generally robust. However, it requires either a very good absorber doublet searching algorithm or a prohibative  amount of visual inspection of absorbers. Here enters the second method; by taking the SNR on a pixel-by-pixel basis, we can determine if, for a given pixel, does $W_{r} < W_{min}$? This method is comparitavely much less computatinally taxing, and requires less human input for visual inspection.

The purpose of this notebook is to elludidate the mathematics behind this method, and help give the reader an intuition on why the equations are structured how they are. We also introduce, for the first time, a variation on the usually used version of the function from the literature. We also go through how errors are handled in a comprehenvise and robust way.

#### NOTE: This notebook will handle only the completeness calculation but this is to be used in the wider context of $dN/dz$ calculations. For the details of how this information connects with that please see the other notebook in the repository.

## Redshift path sensitivity function

We begin with the redshift path sensitivity function. The details of this function are discussed in the accompanying notebook. Here we are only interested in the last term, this is the completeness term.

$$
g(z) = \sum_{i=0}^{n} H(z_{i}-z_{\min}) \, H(z_{\max}-z_{i}) \, 
H\left(W_{\min} - N_{\sigma} \frac{\sigma_{\text{EW}}(z)}{(1+z)}\right)
$$

Where the summation is over n QSO's, H is the Heaviside step function, and $W_{min}$ is an arbitrary equivalent width cutoff, $N_{\sigma}$ is the confidence level (by convention we choose $N_{\sigma}=3$), $z_{i}$ is the redshift of the ith, quasar $z_{min}$ and $z_{max}$ are the minimum and maximum possible search areas for the survey, respectively.

Furthermore, $\sigma_{EW}(z)$ is the ratio of the error and to continuum fitted flux:

$$
    \sigma_{EW}(z)^{2} = \sum_{j}^{npix} \Delta\lambda_{i}^{2} (\sigma_{I_{j}}/I_{j}^{c})^{2}
$$

$\Delta\lambda_{i}$ is determined by the grid size; the distance between pixels, and the summation is over the number of pixels that the equivalent width is being computed over ($npix$). We assume that the continuum fitting error is negligible.

The code determines $npix$ empirically by first finding all detections within the $W_{r}$ range of $W_{min}$. For these detections it then converts the normalized flux to optical depth and finds the velocity range that accounts for between the $1^{st}$ and $99^{th}$ percentile of the cumulative sum of the optical depth. This velocity range is again converted to a number of pixels. This process provides a distribution of typical pixel sizes of detections. Next the code runs a bootstrapping algorithm on these distributions to get a sizable and representative set of $npix$ values. This provides well constrained errors associated with the completeness test, $\sigma_{comp}$.

## Some nuance

The definition commonly used is a $3\sigma$ detection for the primary line in a survey (e.g. $MgII_{2796}$) and a $3\sigma$ detection for a secondary line (e.g. $MgII_{2803}$). This requirement of the secondary line is essential to break degeneracy between the primary line and all other possible transitions. However, in the case of our example, the oscillator strength ratio between the two transitions is approximately 2 (or 0.5 depending on the order). This means that if we were searching for $W_{min} = 0.3\AA$ for $MgII_{2796}$ we would also be looking for $W_{min} = 0.15\AA$ for $MgII_{2803}$. Our completeness correction must take into account cases where we could hypothetically find $MgII_{2796}$ but could not find $MgII_{2803}$ at the corresponding pixels. In short, we are only as complete as our weakest line. In this vein we introduce additional terms to $g(z)$. Here is the modification for our example:

$$
\begin{aligned}
g(z) = \sum_{i=0}^{n} &\ H(z_{i}-z_{\min})\, H(z_{\max}-z_{i}) \\
&\ H\left(W_{\min}^{\text{Mg2796}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Mg2796}}(z)}{(1+z)}\right) \\
&\ H\left(W_{\min}^{\text{Mg2803}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Mg2803}}(z)}{(1+z)}\right)
\end{aligned}
$$


Building on this we introduce a more complex example. When searching for $FeII$ there are four lines that are commonly used: $FeII_{2344}$, $FeII_{2382}$, $FeII_{2586}$, $FeII_{2600}$. Here the definition of a detection remains a $3\sigma$ detection of the primary line and a $3\sigma$ detection of a secondary line, but now we have several secondary lines and we only require a detection of one of them. Therefore we introduce this modification to $g(z)$:

$$
\begin{aligned}
g(z) = \sum_{i=0}^{n} &\ H(z_{i}-z_{\min})\, H(z_{\max}-z_{i}) \\
&\ H\left(W_{\min}^{\text{Fe2600}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Fe2600}}(z)}{(1+z)}\right) \\
&\ \left[ H\left(W_{\min}^{\text{Fe2586}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Fe2586}}(z)}{(1+z)}\right) \right. \\
&\ \quad \left. \lor H\left(W_{\min}^{\text{Fe2383}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Fe2383}}(z)}{(1+z)}\right) \right. \\
&\ \quad \left. \lor H\left(W_{\min}^{\text{Fe2344}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Fe2344}}(z)}{(1+z)}\right) \right]
\end{aligned}
$$

Where $\lor$ is the logical 'OR' statement.

Finally, we think it instructive to give this specific example. There are redshift ranges where the available lines for a $FeII$ survey are $FeII_{2586}$, $FeII_{2600}$. However, $FeII_{2586}$ has a very weak oscillator strength and therefore the completeness of will be very low, and practically speaking there would not be many detections. In the limit where the completeness is very low the values become unreliable and a small variation in completeness causes large variations in the number of absorbers after the correction. In this case it is useful to allow a stronger line to act as an additional secondary line, such as $MgII_{2796}$. This gives $g(z)$ of this form:

$$
\begin{aligned}
g(z) = \sum_{i=0}^{n} &\ H(z_{i} - z_{\min})\, H(z_{\max} - z_{i}) \\
&\ H\left(W_{\min}^{\text{Fe2600}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Fe2600}}(z)}{1+z}\right) \\
&\ \left[ H\left(W_{\min}^{\text{Fe2586}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Fe2586}}(z)}{1+z}\right) \right. \\
&\ \quad \left. \lor H\left(W_{\min}^{\text{Mg2796}} - N_{\sigma} \frac{\sigma_{\text{EW}}^{\text{Mg2796}}(z)}{1+z}\right) \right]
\end{aligned}
$$


We hope that this notebook gives good insight into these modifications and that it is clear to the reader why completeness corrections are a vital part of astrophysical surveys.